In [1]:
import numpy as np
import tensorflow as tf
import cv2
import math

## Conv net hyper parameters

In [2]:
img_size = 100
num_channels = 1
num_classes = 2

filter_size1 = 3
num_filters1 = 32

filter_size2 = 3
num_filters2 = 64

filter_size3 = 3
num_filters3 = 128

filter_size4 = 3
num_filters4 = 128

fc_layer = 256

lr=0.001

## Conv layer

In [3]:
def create_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    
    filter_shape = [filter_size, filter_size, num_input_channels, num_filters]
    
    print(filter_shape)
    
    weights = tf.Variable(tf.truncated_normal(shape=filter_shape))
    
    biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))
    
    layer = tf.nn.conv2d(input=input,
                        filter=weights,
                        strides=[1, 1, 1, 1],
                         padding='SAME')
    
    layer += biases
    
    if use_pooling:
        
        layer = tf.nn.max_pool(value=layer,
                              ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
        
    layer = tf.nn.relu(layer)
    
    return layer, weights

## FC layer

In [4]:
def create_fully_connected(input,
                          num_inputs,
                          num_outputs,
                          use_relu=True):
    
    weights = tf.Variable(tf.truncated_normal(shape=[num_inputs, num_outputs]))
    
    biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
    
    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
        
    return layer

## Flatten conv layer

In [5]:
def flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    
    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features

## Placeholders

In [6]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='y')

## Conv 1

In [7]:
layer_conv1, weights_conv1 = create_conv_layer(input=x,
                                              num_input_channels=num_channels,
                                              filter_size=filter_size1,
                                              num_filters=num_filters1,
                                              use_pooling=True)

[3, 3, 1, 32]


In [8]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 50, 50, 32) dtype=float32>

## Conv 2

In [9]:
layer_conv2, weights_conv2 = create_conv_layer(input=layer_conv1,
                                              num_input_channels=num_filters1,
                                              filter_size=filter_size2,
                                              num_filters=num_filters2,
                                              use_pooling=True)

[3, 3, 32, 64]


In [10]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 25, 25, 64) dtype=float32>

## Conv 3

In [11]:
layer_conv3, weights_conv3 = create_conv_layer(input=layer_conv2,
                                              num_input_channels=num_filters2,
                                              filter_size=filter_size3,
                                              num_filters=num_filters3,
                                              use_pooling=True)

[3, 3, 64, 128]


In [12]:
layer_conv3

<tf.Tensor 'Relu_2:0' shape=(?, 13, 13, 128) dtype=float32>

## Conv 4

In [13]:
layer_conv4, weights_conv4 = create_conv_layer(input=layer_conv3,
                                              num_input_channels=num_filters3,
                                              filter_size=filter_size4,
                                              num_filters=num_filters4,
                                              use_pooling=True)

[3, 3, 128, 128]


In [14]:
layer_conv4

<tf.Tensor 'Relu_3:0' shape=(?, 7, 7, 128) dtype=float32>

## FC1

In [15]:
layer_flat, num_features = flatten_layer(layer_conv4)
print(f"Num features: {num_features}")

Num features: 6272


In [16]:
fc_layer1 = create_fully_connected(input=layer_flat,
                                  num_inputs=num_features,
                                  num_outputs=fc_layer,
                                  use_relu=True)

In [17]:
fc_layer1

<tf.Tensor 'Relu_4:0' shape=(?, 256) dtype=float32>

## FC2

In [18]:
fc_layer2 = create_fully_connected(input=fc_layer1,
                                  num_inputs=fc_layer,
                                  num_outputs=num_classes,
                                  use_relu=False)

In [19]:
fc_layer2

<tf.Tensor 'add_5:0' shape=(?, 2) dtype=float32>

## Softmax

In [20]:
y_pred = tf.nn.softmax(fc_layer2)
y_pred_class = tf.argmax(y_pred, axis=1)

In [21]:
y_pred_class

<tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>

## Cost

In [22]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc_layer2, labels=y)

In [23]:
cost = tf.reduce_mean(cross_entropy)

## Optimizer

In [24]:
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

## Saver

In [25]:
saver = tf.train.Saver()

In [26]:
save_dir = 'checkpoints/'

## Running Tensorflow session

In [27]:
session = tf.Session()

In [28]:
session.run(tf.global_variables_initializer())

In [29]:
saver.restore(sess=session, save_path=save_dir)

INFO:tensorflow:Restoring parameters from checkpoints/


In [30]:
def resize_image(img, img_size):
    y = math.floor((img_size/img.shape[0])*img.shape[1])
    resized = cv2.resize(img, (y, img_size))
    return resized

def crop_edges(img, final_width):
    left_border = math.floor((img.shape[1] - final_width) / 2)
    right_border = final_width + left_border
#     print(left_border)
#     print(right_border)
    img_cropped = img[:, left_border:right_border]
    return img_cropped


In [31]:
cap = cv2.VideoCapture(0)

In [32]:
image_size = 100
recognition_ratio = 5

In [41]:

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_img = resize_image(gray, image_size)
    cropped_img = crop_edges(resized_img, image_size).reshape(1, img_size, img_size, 1)
#     print(cropped_img.shape)
    pred = session.run(y_pred_class, feed_dict={x: cropped_img})
    print(pred)
#     cv2.imshow('RAW' ,frame)
#     cv2.imshow('PROCESSED', cropped_img)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
# cap.release()
# cv2.destroyAllWindows()

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


KeyboardInterrupt: 

In [34]:
saver = tf.train.Saver()
save_dir = "checkpoints/"